In [1]:
%matplotlib inline
import os
import sys
import numpy as np

sys.path.append("../../")

from sapsan.lib.backends.fake import FakeBackend
from sapsan.lib.backends.mlflow import MLflowBackend
from sapsan.lib.data.hdf5_dataset import HDF5Dataset
from sapsan.lib.data import EquidistanceSampling
from sapsan.lib.estimator import KRR, KRRConfig
from sapsan.lib.experiments.evaluate_3d import Evaluate3d
from sapsan.lib.experiments.evaluate_flatten import EvaluateFlatten
from sapsan.lib.experiments.train import Train

/home/pkarpov/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_least_angle.py:34: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
/home/pkarpov/anaconda3/lib/python3.7/site-packages/catalyst/tools/registry.py:45: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  class Registry(collections.MutableMapping):
/home/pkarpov/anaconda3/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:572: DeprecationWarning:

`np.object` is a deprecated alias for the builtin `object`. 

In [ ]:
#--- Experiment tracking backend ---
experiment_name = "KRR experiment"

#Fake (disable backend)
#tracking_backend = FakeBackend(experiment_name)

#MLflow
#launch mlflow with: mlflow ui --port=9000
#uncomment tracking_backend to use mlflow

MLFLOW_HOST = "localhost"
MLFLOW_PORT = 9000

tracking_backend = MLflowBackend(experiment_name, MLFLOW_HOST, MLFLOW_PORT)

In [ ]:
#--- Data setup ---
#In the intereset of loading and training multiple timesteps
#one can specify which checkpoints to use and where
#they appear in the path via syntax: {checkpoint:format}
#
#Next, you need to specify which features to load; let's assume 
#        path = "{feature}.h5"
#
# 1) If in different files, then specify features directly;
#    The default HDF5 label will be the last label in the file
#    Ex: features = ['velocity', 'denisty', 'pressure']
# 2) If in the same, then duplicate the name in features
#    and specify which labels to pull
#    Ex: features = ["data", "data", "data"]
#        feature_labels = ['velocity', 'density', 'pressure']

path = 'data/t{checkpoint:1.0f}/{feature}_dim128_2d.h5'
features = ['u']
target = ['u']

#dimensionality of the data
AXIS = 2

#Dimensionality of your data per axis
CHECKPOINT_DATA_SIZE = 128

#Reduce dimensionality of each axis to
SAMPLE_TO = 64

#Sampler to use for reduction
sampler = EquidistanceSampling(CHECKPOINT_DATA_SIZE, SAMPLE_TO, AXIS)

In [ ]:
#--- Data setup ---
#In the intereset of loading and training multiple timesteps
#one can specify which checkpoints to use and where
#they appear in the path via syntax: {checkpoint:format}
#
#Next, you need to specify which features to load; let's assume 
#        path = "{feature}.h5"
#
# 1) If in different files, then specify features directly;
#    The default HDF5 label will be the last label in the file
#    Ex: features = ['velocity', 'denisty', 'pressure']
# 2) If in the same, then duplicate the name in features
#    and specify which labels to pull
#    Ex: features = ["data", "data", "data"]
#        feature_labels = ['velocity', 'density', 'pressure']

path = "data/t{checkpoint:1.0f}/{feature}_dim32_fm15.h5"
features = ['u']
target = ['u']

#dimensionality of the data
AXIS = 3

#Dimensionality of your data per axis
CHECKPOINT_DATA_SIZE = 32

#Reduce dimensionality of each axis to
SAMPLE_TO = 16

#Sampler to use for reduction
sampler = EquidistanceSampling(CHECKPOINT_DATA_SIZE, SAMPLE_TO, AXIS)

In [ ]:
#--- Train the model ---
#Machine Learning model to use

#Configuration of the model parameters:
#    alpha - rbf kernel hyperparameter
#    gamma - rbf kernel hyperparameter
estimator = KRR(
    config=KRRConfig(gamma=1.778, alpha=0.001)
)

#Load the data
data_loader = HDF5Dataset(path=path,
                   features=features,
                   target=target,
                   checkpoints=[0],
                   checkpoint_data_size=CHECKPOINT_DATA_SIZE,
                   sampler=sampler, 
                   axis = AXIS,
                   flat = True)
x, y = data_loader.load()

#ex: predict one component of the target
y = np.array([y[0]])

#Set the experiment
training_experiment = Train(name=experiment_name,
                             backend=tracking_backend,
                             model=estimator,
                             inputs=x, targets=y,
                             data_parameters = data_loader.get_parameters())
#Train the model
training_experiment.run()

In [2]:
#--- Test the model ---
#Load the test data
data_loader = HDF5Dataset(path=path,
                   features=features,
                   target=target,
                   checkpoints=[0],
                   checkpoint_data_size=CHECKPOINT_DATA_SIZE,
                   sampler=sampler, 
                   axis = AXIS,
                   flat = True)
x, y = data_loader.load()

#ex: predict one component of the target
y = np.array([y[0]])

#Set the test experiment
evaluation_experiment = EvaluateFlatten(name=experiment_name,
                                           backend=tracking_backend,
                                           model=training_experiment.model,
                                           inputs=x, targets=y,
                                           checkpoint_data_size=SAMPLE_TO,
                                           checkpoints=[0], 
                                           axis = AXIS)

#Test the model
evaluation_experiment.run()

NameError: name 'path' is not defined